In [1]:
from copy import deepcopy

class FloramOQueue: # one level piece only
    def __init__(self, level, levelCount, foq):
        self.level = level
        self.elementCount = 2 ** self.level
        self.up = [[0] * self.elementCount for _ in range(3)]
        self.down = [[0] * self.elementCount for _ in range(3)]
        self.head = -1
        self.tail = -1

        if self.level == levelCount - 1 or foq is None:
            self.next = None
        else:
            self.next = foq[self.level + 1]

    def foq_push(self, value, levelCount, foq):
        self.tail += 1
        self.down[self.tail] = value
        if self.head == 2 and self.tail == 1 and self.next is None:
            levelCount += 1
            temp = FloramOQueue(self.level + 1, levelCount, foq)
            foq.append(temp)
            self.next = foq[-1]
            
        if self.next is None:
            if self.tail == 2 and self.head == -1:
                    temp = deepcopy(self.up)
                    self.up = deepcopy(self.down)
                    self.down = temp
                    self.tail = -1
                    self.head = 2        
        else:
            if self.tail == 2:            
                levelCount = self.next.foq_push(self.down.pop(0) + self.down.pop(0), levelCount, foq) 
                self.down.append([0] * self.elementCount)
                self.down.append([0] * self.elementCount)
                self.tail -= 2
        return levelCount

    def foq_pop(self, levelCount):
        if self.level == levelCount - 1 and self.head == -1 and self.tail != -1:
            temp = deepcopy(self.up)
            self.up = deepcopy(self.down)
            self.head = self.tail
            self.down = temp
            self.tail = -1        
        if self.head <= 0 and self.next:
            v, levelCount = self.next.foq_pop(levelCount)
            self.up[self.head + 1] = v[:self.elementCount]
            self.up[self.head + 2] = v[self.elementCount:]
            self.head += 2
            if self.next.head == -1 and self.next.tail == -1:
                self.next = None
                levelCount -= 1
        value = self.up.pop(0)
        self.up.append([0] * self.elementCount)
        self.head -= 1
        return value, levelCount

    def print_state(self):
        print("Level:", self.level)
        print("Head:", self.head)
        print("Tail:", self.tail)
        print("Up:", self.up)
        print("Down:", self.down)
        print("---------------------------")

levelCount = 1
foq = [FloramOQueue(0, levelCount, None)]
foq[0].print_state()  # 打印初始状态

# 执行push操作
for j in range(100):
    levelCount = foq[0].foq_push([j+1], levelCount, foq)
    i = 0
    while i < levelCount:
        foq[i].print_state()
        i += 1
'''
# 执行pop操作
for j in range(100):
    value, levelCount = foq[0].foq_pop(levelCount)
    print(value)
    i = 0
    while i < levelCount:
        foq[i].print_state()
        i += 1
'''

Level: 0
Head: -1
Tail: -1
Up: [[0], [0], [0]]
Down: [[0], [0], [0]]
---------------------------
Level: 0
Head: -1
Tail: 0
Up: [[0], [0], [0]]
Down: [[1], [0], [0]]
---------------------------
Level: 0
Head: -1
Tail: 1
Up: [[0], [0], [0]]
Down: [[1], [2], [0]]
---------------------------
Level: 0
Head: 2
Tail: -1
Up: [[1], [2], [3]]
Down: [[0], [0], [0]]
---------------------------
Level: 0
Head: 2
Tail: 0
Up: [[1], [2], [3]]
Down: [[4], [0], [0]]
---------------------------
Level: 0
Head: 2
Tail: 1
Up: [[1], [2], [3]]
Down: [[4], [5], [0]]
---------------------------
Level: 1
Head: -1
Tail: -1
Up: [[0, 0], [0, 0], [0, 0]]
Down: [[0, 0], [0, 0], [0, 0]]
---------------------------
Level: 0
Head: 2
Tail: 0
Up: [[1], [2], [3]]
Down: [[6], [0], [0]]
---------------------------
Level: 1
Head: -1
Tail: 0
Up: [[0, 0], [0, 0], [0, 0]]
Down: [[4, 5], [0, 0], [0, 0]]
---------------------------
Level: 0
Head: 2
Tail: 1
Up: [[1], [2], [3]]
Down: [[6], [7], [0]]
---------------------------
Level:

'\n# 执行pop操作\nfor j in range(100):\n    value, levelCount = foq[0].foq_pop(levelCount)\n    print(value)\n    i = 0\n    while i < levelCount:\n        foq[i].print_state()\n        i += 1\n'

In [1]:
def client_compute_path(foq, index):
    levelCount = len(foq)
    ifdown = False
    levelEle = []
    level = 0
    index_cp = deepcopy(index)
    while(level < levelCount):
        levelEle.append((foq[level].head+1) * foq[level].elementCount)
        if index_cp <= levelEle[-1]:
            break
        else:
            index_cp = index_cp - levelEle[-1]
            level = level + 1
            
    if level == levelCount:
        ifdown = True
        level -= 1
        level_mirror = level
        while(level_mirror > 0):
            levelEle.append((foq[level_mirror].tail+1) * foq[level_mirror].elementCount)
            if index_cp <= levelEle[-1]:
                break
            else:
                index_cp = index_cp - levelEle[-1]
                level_mirror = level_mirror - 1
        level = level_mirror
    
    path = bin(index_cp)[2:] #以0b开头
    path = [char == '1' for char in path]
    # 位数不均匀可以考虑补0
    
    return level, ifdown, path

# client_compute_path(foq, 1, levelCount)

In [ ]:
import math
'''
min
3*1 3*2 ... 3*2^(i-1)
1*1 1*2 ... 1*2^(i-1) 1*2^i
= 5*2^i -4
max
3*1 3*2 ... 3*2^i
2*1 2*2 ... 2*2^i
= 5*(2^(i+1) -1)

log2(x/5+1)-1 <= i <= log2((x+4)/5)
'''

def foq_reverse_pop(foq):
    levelCount = len(foq)
    if foq[-1].tail == -1:
        temp = deepcopy(foq[-1].up)
        foq[-1].up = deepcopy(foq[-1].down)
        foq[-1].head = foq[-1].tail
        foq[-1].down = temp
        foq[-1].tail = 2
    if levelCount == 1:
        return
    i = -2
    while foq[i].tail == -1:
        v = foq[i+1].down.pop(0)
        foq[i+1].down.append([0] * foq[i+1].elementCount)
        foq[i+1].tail -= 1
        foq[i].down[foq[i].tail + 1] = v[:foq[i].elementCount]
        foq[i].down[foq[i].tail + 2] = v[foq[i].elementCount:]
        foq[i].tail += 2
        i -= 1
        
def foq_init(data): 
    # 先把up装满，把down从后向前装满，再对down使用pop，对up使用down的push策略(向下翻转)
    # 更加简略：向下翻转只发生在最后一层即可，并可以从最后一层开始向前pop
    datalen = len(data)
    datain = 0
    levelCount = math.ceil(math.log2(datalen/5 +1))-1 +1
    # 或math.floor(math.log2((15+4)/5))，二者相等    
    foq = [0] * levelCount
    for i in range(levelCount):
        foq[levelCount-1-i] = FloramOQueue(levelCount-1-i, levelCount, foq)
    for i in range(levelCount):
        if datalen-datain > 3*foq[i].elementCount:
            for j in range(3):
                foq[i].up[j] = data[datain+foq[i].elementCount*j : datain+foq[i].elementCount*(j+1)]
                foq[i].head = 2
            datain += foq[i].elementCount*3
    for i_ in range(levelCount):
        i = i_ + 1
        for j in range(3):        
            if datalen-datain >= foq[-i].elementCount:
                foq[-i].down[j] = data[datain : datain+foq[-i].elementCount]
                foq[-i].tail += 1
                datain += foq[-i].elementCount
            else:
                break
    foq_reverse_pop(foq)
    return foq, levelCount

'''
data = list(range(1, 101))
foq1, levelCount = foq_init(data)
'''

In [ ]:
def compare_and_swap(arr, i, j, direction):
    if (direction == 1 and arr[i] > arr[j]) or (direction == 0 and arr[i] < arr[j]):
        arr[i], arr[j] = arr[j], arr[i]
def bitonic_merge(arr, low, cnt, direction):
    if cnt > 1:
        k = cnt // 2
        for i in range(low, low + k):
            compare_and_swap(arr, i, i + k, direction)
        bitonic_merge(arr, low, k, direction)
        bitonic_merge(arr, low + k, k, direction)
def bitonic_sort(arr, low, cnt, direction):
    if cnt > 1:
        k = cnt // 2
        bitonic_sort(arr, low, k, 1)
        bitonic_sort(arr, low + k, k, 0)
        bitonic_merge(arr, low, cnt, direction)
    # 1e7个数据，3min43.2s

def foq_init_vector_copy(foq):
    levelCount = len(foq)
    read_copy = [[] * levelCount for _ in range(2)] 
    for i in range(2):
        read_copy[i] = [[] for _ in range(levelCount)]
    # 这种形式可以为穿梭(随机生成需读取index之后的FSS层)批量读取做准备
    # 好处是路径转换开销小且不用获取head和tail，坏处是需要额外一次对levelCount做选择（已证明不能使用FSS，但可以使用集合版秘密共享（levelCount的三分之一次方））
    # 用安全多方计算如何模拟选择需要一排（串联）二路选择器，也就是说可能和levelCount成线性
    # 倒也不是不可以每个level存储在一个ORAM里
    # socket.receive(foq_other)
    
    for i in range(levelCount):
        for x,y in zip(foq[i].up, foq_other[i].up):
            for a,b in zip(x,y):
                read_copy[0][i].append(a^b)
    
    for i in range(levelCount):
        for x,y in zip(foq[levelCount-1 -i].down, foq_other[levelCount-1 -i].down):
            for a,b in zip(x,y):
                read_copy[1][i].append(a^b)
    
    #完全拉直且有序的版本            
    read_copy_flat = []
    for sublist in read_copy[0]:
        read_copy_flat.extend(sublist)
    for sublist in read_copy[1]:
        read_copy_flat.extend(sublist)
    bitonic_sort(read_copy_flat, 0, len(read_copy_flat), 1)

    return read_copy, read_copy_flat